# What is NumPy

**NumPy** (Numerical Python) 是 **Python** 科学计算的基础包。它是一个 **Python** 库，提供了一个多维数组对象、各种派生对象（例如掩码数组和矩阵），以及用于对数组进行快速操作的各种例程，包括**数学、逻辑、形状操作、排序、选择、I/O、离散傅立叶变换、基本线性代数、基本统计运算、随机模拟**等等。

## ADVANTAGE

**NumPy** 的核心是 **ndarray** 对象。其封装了**同构**数据类型的 **n 维数组**，许多操作在编译代码中执行以提高性能。NumPy 数组和标准 Python 序列之间有几个重要的区别：

* 与 Python 列表（可以动态增长）不同，NumPy 数组在创建时具有固定大小。更改 **ndarray** 的大小将创建一个新数组并删除原始数组；
* NumPy 数组中的元素都需要具有相同的数据类型，因此在内存中的大小相同。例外：可以有（Python，包括 NumPy）对象的数组，从而允许不同大小元素的数组；
* NumPy 数组有助于对大量数据进行高级数学运算和其他类型的运算。通常，与使用 Python 的内置序列相比，此类操作的执行效率更高，代码更少。

### [ EXAMPLE ]

大数据时代的来临，使我们在构造模型时必须高度注意算法/程序的内存使用以及算力开销。毋庸置疑，大量的第三方库以及简洁的语法是 Python 的优势，但运行速度绝对是它不可否认的短板。

高级语言是为了方便用户编程而诞生的程序设计语言，计算机是无法直接执行程序指令的，它们需要先被编译为机器语言（0/1代码）才能交给计算机执行。
* Python 作为解释语言，需要解释器逐条解释（词法分析 $\rightarrow$ 语法分析 $\rightarrow$ 语义分析）执行；
* 而 C++ 是典型的编译语言，编译器读取源代码，再输出可执行代码：编译器前端（词法分析 $\rightarrow$ 语法分析 $\rightarrow$ 语义分析 $\rightarrow$ 中间代码生成）$\rightarrow$ 编译器后端（代码优化 $\rightarrow$ 目标代码生成），执行时不再需要编译器，直接在支持目标代码的平台上运行，因此执行效率比解释执行快很多。

这里我们来看一个简单的编程实例：考虑将一维序列中的每个元素，与另一个相同长度序列中的相应元素相乘的情况。如果数据存储在两个 Python 列表 `a` 和 `b` 中，我们可以遍历列表完成元素相乘：

In [5]:
import time

def list_mul(a, b):

    c = []
    for i in range(len(a)):
        c.append(a[i] * b[i])

a = list(range(1000000))
b = list(range(1000000))
cur_t = time.time()
c = list_mul(a, b)
print('The run time is: ' + str(time.time() - cur_t) + 's')

The run time is: 0.15964174270629883s


然后，我们以相同的算法编写 C++ 程序，并编译执行：

```c++
#include <iostream>

using namespace std;

int* listMul(int a[], int b[], int length) {
    int *c = new int[length];
    for (int i=0; i<length; ++i) {
        c[i] = a[i] * b[i];
    }

    return c;
}

int main() {
    int a[1000000], b[1000000] = {0};
    int length = sizeof(a)/sizeof(a[0]);
    for (int i=0; i<length; ++i) {
        a[i] = b[i] = i;
    }

    clock_t start_time=clock();
    int *c = listMul(a, b, length);
    clock_t end_time=clock();
    delete[] c;
    cout << "The run time is: " <<(double)(end_time - start_time) / CLOCKS_PER_SEC << "s" << endl;

    return 0;
}
```

编译运行结果如图：
![图1](/image/listMulResult.jpg)
显然，编程逻辑相同的情况下，C++ 的运行速度会快于 Python。

但是，当我们需要编写更加复杂的算法逻辑时，C++ 所需的编码工作会随着数据维度的增加而增加。此时，NumPy 为我们提供了两全其美的方法：当涉及 **ndarray** 时，逐元素操作是“默认模式”，但逐元素操作由预编译的 C 代码快速执行。在 NumPy 中：

In [2]:
import numpy as np
import time

a = np.array(range(1000000))
b = np.array(range(1000000))

cur_t = time.time()
c = a * b
print('The run time is: ' + str(time.time() - cur_t) + 's')

The run time is: 0.0029189586639404297s


在保留 Python 语言语法简洁、编程逻辑简单的基础上，加快操作数据的速度，这就是 NumPy 正在做的事情！

## WHY NumPy FASTER

NumPy 大部分强大功能的基础是**矢量化**和**广播**：

* **矢量化**描述了代码中没有任何显式循环、索引等。当然，这些事情只是在优化、预编译 C 代码的“幕后”发生的。矢量化代码有很多优点，其中包括：
  * 更简洁易读；
  * 更少的代码往往意味着更少的错误；
  * 代码更接近标准数学符号（通常更容易正确编码数学结构）；
  * 矢量化会产生更多的 `Pythonic` 代码，如果没有矢量化，代码中会包含更多低效且难以阅读的 `for` 循环。
* **广播**是用于描述操作的隐式逐元素行为的术语；一般而言，在 NumPy 中，所有操作，不仅是算术运算，还有逻辑、按位、函数等，都以这种隐式的逐元素方式表现，即它们正在广播。此外，在上面的 [[EXAMPLE]](###[EXAMPLE]) 中，`a` 和 `b` 可以是相同形状的多维数组，或者一个标量和一个数组，甚至两个不同形状的数组，前提是较小的数组可以“扩展”到较大的数组中，由此产生的广播是明确的。

我们再来看一个广播的例子：

In [1]:
import numpy as np

a = np.array(range(10))
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [2]:
a += 1
a

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [19]:
a = np.array([1, 2])
print(a.shape)
b = np.array([[1, 2], [4, 5], [7, 8]])
print(b.shape)

(2,)
(3, 2)


In [20]:
print(a + b)
print((a + b).shape)

[[ 2  4]
 [ 5  7]
 [ 8 10]]
(3, 2)


In [21]:
a = np.ones((2, 5, 3, 4))
print(a.shape)
b = np.zeros((2, 2))
print(b.shape)

(2, 5, 3, 4)
(2, 2)


In [22]:
a + b

ValueError: operands could not be broadcast together with shapes (2,5,3,4) (2,2) 

此时，广播的机制就不再生效，并直接告诉我们 `operands could not be broadcast together with shapes (3,3) (2,2)`，因为此时 `a` 和 `b` 的最后一维不等，不能广播，但如果我们做如下修改：

In [25]:
a = np.ones((2, 5, 3, 4))
print(a.shape)
b = np.zeros((3, 4))
print(b.shape)

(2, 5, 3, 4)
(3, 4)


In [27]:
print((a + b).shape)

(2, 5, 3, 4)


此时就又可以广播了。这里再看一个有趣的广播：

In [28]:
a = np.ones((2, 5, 3, 1))
print(a.shape)
b = np.zeros((5, 1, 4))
print(b.shape)

(2, 5, 3, 1)
(5, 1, 4)


In [29]:
print((a + b).shape)

(2, 5, 3, 4)


这里给出两个广播的规则：

* `a` 和 `b` 各维度大小从后往前进行比较，如果一致则可以广播；
* `a` 和 `b` 各维度大小存在不一致，但不一致的维度中存在一个为 1，则可以广播

## REFERENCES

[1] [Numpy中文教程](https://www.w3cschool.cn/numpy_ln/)